In [ ]:
def inputMessage():

    ''''
    message = input('message: ')
    encrypt_type = int(input('encyption type (number): '))
    encryption_key = input('encryption key: ')
    encryption_dir = input("encrypt/decrypt")
    '''
    message = 'hello world'
    encrypt_type = 1
    encrypt_key = 7
    encrypt_dir = 'e'

    


In [ ]:
#ef encryptMessage(message, encrypt_type, encrypt_key, encrypt_dir):

    



In [ ]:
def ascii_ceasar_encrypt(message):

    


    

In [ ]:
def ascii_convert(msg):

    lst = []
    for i in msg:
        lst.append(ord(i))

    return lst

#print(ascii_convert('hello world'))

[104, 101, 108, 108, 111, 32, 119, 111, 114, 108, 100]


In [8]:
def shift_ascii_numbers(numb_list, shift_val):

    shifted_numb_list =  []
    for i in numb_list:
        temp_val = (i + shift_val) % 255
        shifted_numb_list.append(temp_val)
    return shifted_numb_list

lst = shift_ascii_numbers([104, 101, 108, 108, 111, 32, 119, 111, 114, 108, 254], 7)
lst


[111, 108, 115, 115, 118, 39, 126, 118, 121, 115, 6]